In [ ]:
import requests as rq #网络请求基本库
from bs4 import BeautifulSoup # 清洗网页数据的库
import re # 正则表达式的库
    
# 在新笔趣阁，爬一篇名叫《临高启明》的小说第一章
if __name__ == '__main__':
    target = "https://www.xbiquge.la/0/858/708546.html" # 网址
    req = rq.get(url = target) # 用requests获取网页
    req.encoding = 'utf-8'# 设置网页编码
    html = req.text # 网页的html文件
    # print(html)
    con = BeautifulSoup(html,'lxml') # 利用soup库转换
    res = con.find('div',id='content') # 进行数据筛选
    # for p in res:
    #     print(p)
    res = re.split('(\xa0\xa0\xa0\xa0|\r)',res.text) # 分割文本
    ans=[]
    for x in res: # 筛选文本
        if(x!='\r' and x!='\xa0'*4 and x!='' and x!=' '):
            ans.append(x)
    print(ans)

In [5]:
# 更简单的一个例子
import requests as rq
if __name__ == '__main__':
    website = 'https://www.baidu.com'
    req1 = rq.get(url = website)
    req1.encoding = 'utf-8'
    print(req1.text)


<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/cache/bdorz/baidu.min.css><title>百度一下，你就知道</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlength=255 autocomplete=off autofocus=autofocus></span><span class="bg s_btn